In [1]:
import { ChatOpenAI } from "@langchain/openai"
import zodToJsonSchema from "zod-to-json-schema"

In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 0,
  modelName: "qwen-plus",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}

使用 Zod 辅助 llm 进行参数类型检查

In [3]:
import { z } from "zod"

const stringSchema = z.string()
stringSchema.parse("Hello, Zod!")

"Hello, Zod!"

In [4]:
stringSchema.parse(2323);

ZodError: [
  {
    "code": "invalid_type",
    "expected": "string",
    "received": "number",
    "path": [],
    "message": "Expected string, received number"
  }
]

In [ ]:
const stringSchema = z.string()
const numberSchema = z.number()
const booleanSchema = z.boolean()

const stringArraySchema = z.array(z.string())
stringArraySchema.parse(["apple", "banana", "cherry"])

const personSchema = z.object({
  name: z.string(),
  age: z.number(),
  isStudent: z.boolean().optional(),
  home: z.string().default("no home"),
})

const mixedTypeSchema = z.union([z.string(), z.number()])
mixedTypeSchema.parse("hello")
mixedTypeSchema.parse(42)

In [5]:
const getCurrentWeatherSchema = z.object({
  location: z.string().describe("The city and state, e.g. San Francisco, CA"),
  unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
})

In [6]:
import {zodToJsonSchema} from "zod-to-json-schema"

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema)
console.log(paramSchema)

{
  type: "object",
  properties: {
    location: {
      type: "string",
      description: "The city and state, e.g. San Francisco, CA"
    },
    unit: {
      type: "string",
      enum: [ "celsius", "fahrenheit" ],
      description: "The unit of temperature"
    }
  },
  required: [ "location", "unit" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [7]:
const model = new ChatOpenAI(chatOptions)

const modelWithTools = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: zodToJsonSchema(getCurrentWeatherSchema),
      },
    },
  ],
})

await modelWithTools.invoke("北京天气怎么样")

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          function: [Object],
          index: 0,
          id: "call_8304de84bb22437e9050ed",
          type: "function"
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "北京", "unit": "celsius"}'
        },
        index: 0,
        id: "call_8304de84bb22437e9050ed",
        type: "function"
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 24, promptTokens: 236, totalTokens: 260 },
    finish_reason: "tool_calls"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [8]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["human", "{input}"],
])

const chain = prompt.pipe(modelWithTools)

const response = await chain.invoke({ input: "北京天气怎么样" })
console.log(response)


AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          function: [Object],
          index: 0,
          id: "call_71600c2d49164fcabee0fd",
          type: "function"
        }
      ]
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        function: {
          name: "getCurrentWeather",
          arguments: '{"location": "北京", "unit": "celsius"}'
        },
        index: 0,
        id: "call_71600c2d49164fcabee0fd",
        type: "function"
      }
    ]
  },
  response_metadata: {
    tokenUsage: { completionTokens: 24, promptTokens: 241, totalTokens: 265 },
    finish_reason: "tool_calls"
  },
  tool_calls: [],
  invalid_tool_calls: []
}


多 tools model

In [9]:
const getCurrentTimeSchema = z.object({
  format: z.enum(["ios", "locale", "string"])
  .optional()
  .describe("The format of the time, e.g. iso, locale, string"),
})

zodToJsonSchema(getCurrentTimeSchema)

{
  type: "object",
  properties: {
    format: {
      type: "string",
      enum: [ "ios", "locale", "string" ],
      description: "The format of the time, e.g. iso, locale, string"
    }
  },
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}

In [10]:
const model = new ChatOpenAI(chatOptions)

const modelWithMultiTools = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: zodToJsonSchema(getCurrentWeatherSchema),
      },
    },
    {
      type: "function",
      function: {
        name: "getCurrentTime",
        description: "Get the current time in a given format",
        parameters: zodToJsonSchema(getCurrentTimeSchema),
      },
    },
  ],
})


使用 tools 给数据打标签

In [11]:
const taggingSchema = z.object({
  emotion:z.enum(["pos", "neg", "neutral"]).describe("文本的情感"),
  language: z.string().describe("文本的核心语言（应为ISO 639-1代码）"),
});

In [12]:
const model = new ChatOpenAI(chatOptions)

const modelTagging = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "tagging",
        description: "为特定的文本片段打上标签",
        parameters: zodToJsonSchema(taggingSchema),
      },
    },
  ],
  tool_choice: {
    type: "function",
    function: {
      name: "tagging",
    },
  },
})


In [13]:
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools"

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "仔细思考，你有充足的时间进行严谨的思考，然后按照指示对文本进行标记",
  ],
  ["human", "{input}"],
])

const chain = prompt.pipe(modelTagging).pipe(new JsonOutputToolsParser())

In [ ]:
const res = await chain.invoke({
  input: "hello world"
})

console.log(res)

const res1 = await chain.invoke({
  input: "写代码太难了，👴 不干了"
})

console.log(res1)

const res2 = await chain.invoke({
  // 日语，圣诞快乐
  input: "メリークリスマス!"
})

console.log(res2)

const res3 = await chain.invoke({
  input: "我非常喜欢 AI，特别是 LLM，因为它非常 powerful"
})

console.log(res3)

使用 tools 进行信息提取

In [14]:
const personExtractionSchema = z.object({
  name: z.string().describe("人的名字"),
  age: z.number().optional().describe("人的年龄")
}).describe("提取关于一个人的信息");

In [16]:
const relationExtractSchema = z.object({
  people: z.array(personExtractionSchema).describe("提取所有人"),
  relation: z.string().describe("人之间的关系, 尽量简洁")
})

In [17]:
const schema = zodToJsonSchema(relationExtractSchema)
console.log(schema)

{
  type: "object",
  properties: {
    people: {
      type: "array",
      items: {
        type: "object",
        properties: { name: [Object], age: [Object] },
        required: [ "name" ],
        additionalProperties: false,
        description: "提取关于一个人的信息"
      },
      description: "提取所有人"
    },
    relation: { type: "string", description: "人之间的关系, 尽量简洁" }
  },
  required: [ "people", "relation" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


In [18]:
const model = new ChatOpenAI(chatOptions)

const modelExtract = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "relationExtract",
        description: "提取数据中人的信息和人的关系",
        parameters: zodToJsonSchema(relationExtractSchema),
      },
    },
  ],
  tool_choice: {
    type: "function",
    function: {
      name: "relationExtract",
    },
  },
})

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "仔细思考，你有充足的时间进行严谨的思考，然后提取文中的相关信息，如果没有明确提供，请不要猜测，可以仅提取部分信息",
  ],
  ["human", "{input}"],
])

const chain = prompt.pipe(modelExtract).pipe(new JsonOutputToolsParser())

In [19]:
const res = await chain.invoke({
  input: "小明现在 18 岁了，她妈妈是小丽"
})

console.log(res)

[
  {
    type: "relationExtract",
    args: {
      people: [ { name: "小明", age: 18 }, { name: "小丽" } ],
      relation: "小明是小丽的儿子"
    }
  }
]


In [ ]:
const res = await chain.invoke({
  input: "我是小明现在 18 岁了，我和小 A、小 B 是好朋友，都一样大"
})

console.log(res)

In [20]:
const res = await chain.invoke({
  input: "我是小明"
})
console.log(res)

[
  {
    type: "relationExtract",
    args: { people: [ { name: "小明" } ], relation: "" }
  }
]
